In [1]:
using TuringModels
using Turing

Turing.setadbackend(:reverse_diff)
#nbTuring.turnprogress(false);

d = CSV.read(rel_path("..", "data", "UCBadmit.csv"), delim=';');
size(d) # Should be 12x5

(12, 5)

Turing model

In [2]:
@model m11_5(admit, applications) = begin
    N=length(applications)
    θ ~ Truncated(Exponential(1), 0, Inf)
    α ~ Normal(0,2)

    for i ∈ 1:N
        prob = logistic(α)
        # alpha and beta for the BetaBinomial must be provided.
        # The two parameterizations are related by
        # alpha = prob * theta, and beta = (1-prob) * theta.
        # See https://github.com/rmcelreath/rethinking/blob/master/man/dbetabinom.Rd
        alpha = prob * θ
        beta = (1 - prob) * θ

        admit[i] ~ BetaBinomial(applications[i], alpha, beta)
    end
end

m11_5 (generic function with 3 methods)

Sample

In [3]:
posterior = sample(m11_5(d[:admit],d[:applications]), Turing.NUTS(4000, 1000, 0.9));

┌ Info: [Turing] looking for good initial eps...
└ @ Turing.Inference /Users/rob/.julia/packages/Turing/UXxKz/src/inference/support/hmc_core.jl:240
[NUTS{Turing.Core.FluxTrackerAD,Union{}}] found initial ϵ: 0.1
└ @ Turing.Inference /Users/rob/.julia/packages/Turing/UXxKz/src/inference/support/hmc_core.jl:235
┌ Warning: Numerical error has been found in gradients.
└ @ Turing.Core /Users/rob/.julia/packages/Turing/UXxKz/src/core/ad.jl:168
┌ Warning: grad = [NaN, NaN]
└ @ Turing.Core /Users/rob/.julia/packages/Turing/UXxKz/src/core/ad.jl:169
┌ Info:  Adapted ϵ = 0.22640043955822706, std = [1.0, 1.0]; 1000 iterations is used for adaption.
└ @ Turing.Inference /Users/rob/.julia/packages/Turing/UXxKz/src/inference/adapt/adapt.jl:91
[NUTS] Finished with
  Running time        = 23.116293342999985;
  #lf / sample        = 0.0;
  #evals / sample     = 14.90025;
  pre-cond. metric    = [1.0, 1.0].


Draw summary

In [4]:
describe(posterior)

Iterations = 1:4000
Thinning interval = 1
Chains = 1
Samples per chain = 4000

Empirical Posterior Estimates:
              Mean          SD        Naive SE        MCSE        ESS   
       α  -0.378635554 0.314620755 0.00497459093 0.00559572502 3161.2731
  lf_num   0.000000000 0.000000000 0.00000000000 0.00000000000       NaN
 elapsed   0.005779073 0.013603013 0.00021508252 0.00025877364 2763.3134
 epsilon   0.237155487 0.089244319 0.00141107659 0.00309149988  833.3409
      lp -71.080908202 1.637159578 0.02588576579 0.04913323253 1110.2770
eval_num  14.900250000 7.980834754 0.12618807726 0.16926556570 2223.1021
       θ   2.765211316 0.936177529 0.01480226643 0.01708023688 3004.1986
  lf_eps   0.237155487 0.089244319 0.00141107659 0.00309149988  833.3409

Quantiles:
               2.5%          25.0%          50.0%         75.0%         97.5%    
       α  -0.96982173790  -0.584962094  -0.3839362530  -0.174712537   0.245038538
  lf_num   0.00000000000   0.000000000   0.0000000000   0

Result rethinking

In [5]:
m115rethinking = "
         mean   sd  5.5% 94.5% n_eff Rhat
theta  2.74 0.96  1.43  4.37  3583    1
a       -0.37 0.31 -0.87  0.12  3210    1
";

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*